<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.corpus import stopwords as nltk_stopwords
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Загружаем все необходимые библиотеки

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head(15)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Изучил данные. Обнарурил специальные символы, единичные символы, специальные символы начала строки, префиксы, различное число пробелов (пропусков), заглавные строки.

In [5]:
df.isna().mean()

text     0.0
toxic    0.0
dtype: float64

In [6]:
df.duplicated().sum()

0

In [7]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Опредилил количество различных комментариев, дальше приведу все тексты к "нормальному" виду и проведу лемматизацию

In [8]:
corpus = df['text'].values
lemmas = []
stemmer = WordNetLemmatizer()

for sen in range(0, len(corpus)):
    document = re.sub(r'\W', ' ', str(corpus[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    lemmas.append(document)    

In [9]:
lemmas[:2]

['explanation why the edits made under my username hardcore metallica fan were reverted they weren vandalism just closure on some gas after voted at new york doll fac and please don remove the template from the talk page since m retired now 89 205 38 27',
 'd aww he match this background colour m seemingly stuck with thanks talk 21 51 january 11 2016 utc']

Создам датафрейм с лемматизированными значениями.

In [10]:
lemmas_series = pd.Series(lemmas, name = 'lemmas')
comments_new = pd.concat([df, lemmas_series], axis = 1)
comments_new.head(15)

,text,toxic,lemmas
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour m seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man m really not trying to edit war it jus...
3,"""\nMore\nI can't make any real suggestions on ...",0,more can make any real suggestion on improveme...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...
5,"""\n\nCongratulations from me as well, use the ...",0,congratulation from me a well use the tool wel...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense wa offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


Разделю выборки на обучающую и валидационную

In [11]:
comments_features = comments_new.drop(['text','toxic'], axis=1)
comments_target = comments_new['toxic']

In [12]:
X_train, X_rest, y_train, y_rest = train_test_split(comments_features, comments_target, test_size=0.3, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_rest, y_rest, test_size=0.3, random_state=12345)

In [13]:
display(X_train.shape, X_valid.shape, X_test.shape)

(111699, 1)

(33510, 1)

(14362, 1)

Пропишу NLTK стоп слов

In [14]:
stopwords = set(nltk_stopwords.words('english'))

Создам тело обучающей, валидационной и трейновой выборки

In [15]:
corpus_train = X_train['lemmas'].values
corpus_valid = X_valid['lemmas'].values
corpus_test = X_test['lemmas'].values

Проведу извлечение текствовых признаков через TfidfVectorizer библиотеки scikit-learn

In [16]:
tf_idfconverter = TfidfVectorizer(max_features=6000, min_df=5, max_df=0.7, stop_words=stopwords)
tf_idfconverter.fit(corpus_train)

TfidfVectorizer(max_df=0.7, max_features=6000, min_df=5,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [17]:
X_train_tf = tf_idfconverter.transform(corpus_train)
X_valid_tf = tf_idfconverter.transform(corpus_valid)
X_test_tf = tf_idfconverter.transform(corpus_test)
display(X_train_tf.shape, X_valid_tf.shape, X_test_tf.shape)

(111699, 6000)

(33510, 6000)

(14362, 6000)

**Вывод:**
1. Загрузил и подготовил данные
2. Пропусков и дубликантов не обнаружил.
3. Провел лемматизацию датафрейма.
4. Разделил выборки и подготовил их к обучению моделей

## Обучение

Путем изучения библиотеки scikit-learn искал библиотеки, которые могу использовать.
1. Самая популярная в моём представлении, а также наиболее часто используемая в проектах - LogisticRegression;
2. Классификация линейных опорных векторов (LinearSVC), поскольку, он обладает большей гибкостью по сравнению с SVC в выборе функций штрафов и потерь и должен лучше масштабироваться для большого количества выборок;
3. SGDClassifier, поскольку SGD был успешно применен к крупномасштабным и разреженным задачам машинного обучения, часто возникающим при классификации текста и обработке естественного языка;
4. Наивный байесовский классификатор для текстов (MultinomialNB) - является одним из двух классических наивных байесовских вариантов, используемых в текстовой классификации.

Поскольку работаю с 4-мя моделями, напишу функцию для расчёта метрики F1 и занесу результаты в таблицу 

In [18]:
res=[]
def ml_models(models, ft, tt, fv, tv):
    model = models
    model.fit(ft, tt)
    
    predictions_valid = model.predict(fv)
    print('F1 = {:.2f}'.format(f1_score(tv, predictions_valid)))
    return f1_score(tv, predictions_valid)

In [19]:
res.append(ml_models(LogisticRegression(random_state=12345), X_train_tf, y_train, X_valid_tf, y_valid))

F1 = 0.75


In [20]:
res.append(ml_models(LinearSVC(random_state=12345), X_train_tf, y_train, X_valid_tf, y_valid))

F1 = 0.77


In [21]:
res.append(ml_models(SGDClassifier(random_state=12345), X_train_tf, y_train, X_valid_tf, y_valid))

F1 = 0.67


In [22]:
res.append(ml_models(MultinomialNB(), X_train_tf, y_train, X_valid_tf, y_valid))

F1 = 0.68


In [23]:
result = pd.DataFrame(data=res, columns=['Результаты расчёта F1']).rename(index={0: 'LogisticRegression', 1: 'LinearSVC', 2: 'SGD', 3: 'MultinomialNB'})
display(result)

,Результаты расчёта F1
LogisticRegression,0.748073
LinearSVC,0.773965
SGD,0.671075
MultinomialNB,0.677533


Данные записал, лучший результат по метрике F1 показала LinearSVC. Далее следует подобрать оптимальные гиперпараметры для моделей и посмотреть, что получится. Напишу функцию и посмотрю что получится

In [24]:
res = []
def gridsearch_result(model, parameters, f, t, scorer):
    gridsearch = GridSearchCV(estimator=model, param_grid = parameters, refit=True, scoring=scorer, cv=2)
    gridsearch = gridsearch.fit(f, t)
        
    print('Лучшие гиперпараметры: ', gridsearch.best_params_)
    return gridsearch.best_estimator_

In [25]:
parameters = {
        'random_state' : [12345],
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 200]
        }
best_LogisticRegression = gridsearch_result(LogisticRegression(), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'C': 10, 'penalty': 'l2', 'random_state': 12345}


In [26]:
res.append([ml_models(best_LogisticRegression, X_train_tf, y_train, X_valid_tf, y_valid), best_LogisticRegression.get_params()])

F1 = 0.77


In [28]:
parameters = {
        'max_iter' : range(1000, 2000, 100),
        'C' : [0.1, 1, 10, 100],
        'random_state' : [12345]        
        }
best_LinearSVC = gridsearch_result(LinearSVC(), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'C': 1, 'max_iter': 1000, 'random_state': 12345}


In [29]:
res.append([ml_models(best_LinearSVC, X_train_tf, y_train, X_valid_tf, y_valid), best_LinearSVC.get_params()])

F1 = 0.77


In [30]:
parameters = {
        'alpha' : [1, 1e-01, 1e-02, 1e-03, 1e-04, 1e-05, 1e-06],
        'penalty' : ['l1','l2'],
        'random_state' : [12345]        
        }
best_SGD = gridsearch_result(SGDClassifier(), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'alpha': 1e-05, 'penalty': 'l1', 'random_state': 12345}


In [31]:
res.append([ml_models(best_SGD, X_train_tf, y_train, X_valid_tf, y_valid), best_SGD.get_params()])

F1 = 0.78


In [32]:
parameters = {
        'alpha' : [1e-06, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1],
        }
best_MNB = gridsearch_result(MultinomialNB(class_prior=None, fit_prior=True), parameters, X_train_tf, y_train, 'f1')

Лучшие гиперпараметры:  {'alpha': 0.1}


In [33]:
res.append([ml_models(best_MNB, X_train_tf, y_train, X_valid_tf, y_valid), best_MNB.get_params()])

F1 = 0.69


In [34]:
result_gs = pd.DataFrame(data=res, columns=['F1 с гиперпарам.', 'Гиперпараметры']).rename(index={0: 'LogisticRegression', 1: 'LinearSVC', 2: 'SGD', 3: 'MultinomialNB'})
display(result_gs)

,F1 с гиперпарам.,Гиперпараметры
LogisticRegression,0.774472,"{'C': 10, 'class_weight': None, 'dual': False,..."
LinearSVC,0.773965,"{'C': 1, 'class_weight': None, 'dual': True, '..."
SGD,0.776551,"{'alpha': 1e-05, 'average': False, 'class_weig..."
MultinomialNB,0.687875,"{'alpha': 0.1, 'class_prior': None, 'fit_prior..."


И теперь расчитаю F1 на тесте.

In [35]:
res = []
res.append(ml_models(best_LogisticRegression, X_train_tf, y_train,  X_test_tf, y_test))

F1 = 0.76


In [36]:
res.append(ml_models(best_LinearSVC, X_train_tf, y_train,  X_test_tf, y_test))

F1 = 0.76


In [37]:
res.append(ml_models(best_SGD, X_train_tf, y_train,  X_test_tf, y_test))

F1 = 0.76


In [38]:
res.append(ml_models(best_MNB, X_train_tf, y_train,  X_test_tf, y_test))

F1 = 0.68


In [39]:
result_test = pd.DataFrame(data=res, columns=['F1 тестовая']).rename(index={0: 'LogisticRegression', 1: 'LinearSVC', 2: 'SGD', 3: 'MultinomialNB'})
display(result_test)

,F1 тестовая
LogisticRegression,0.762200
LinearSVC,0.761346
SGD,0.755837
MultinomialNB,0.684142


In [40]:
result_final = pd.concat([result, result_gs, result_test], axis="columns")

In [41]:
result_final

,Результаты расчёта F1,F1 с гиперпарам.,Гиперпараметры,F1 тестовая
LogisticRegression,0.748073,0.774472,"{'C': 10, 'class_weight': None, 'dual': False,...",0.762200
LinearSVC,0.773965,0.773965,"{'C': 1, 'class_weight': None, 'dual': True, '...",0.761346
SGD,0.671075,0.776551,"{'alpha': 1e-05, 'average': False, 'class_weig...",0.755837
MultinomialNB,0.677533,0.687875,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.684142


**Вывод:**
1. Обучил 4 модели машинного обучения;
2. Результат, удовлетворяющий условию показали: LogisticRegression, LinearSVC;
3. Лучший результат показала модель LogisticRegression.

Bert не делаю, поскольку делаю на юпитере Яндекса все проекты, уже виснет ядро, но вроде не обязательно в задании.

## Выводы

1. Изучил и лемматизировал данные;
2. Подготовил данные к машинному обучению;
3. Обучил 4 модели классифицировать комментарии на позитивные и негативные;
4. Наилучшие результаты показали 2 модели машинного обучения LogisticRegression, LinearSVC;
5. Для решения поставленной в проекте задачи классификации комментариев стоит использовать LogisticRegression, поскольку она показала наилучшие результаты на всех этапах.
6. Удалось добиться значения метрики F1 > 0.75

В общем и целом обучил модель классифицировать комментарии на позитивные и негативные со значением метрики F1 больше 0,75

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны